In [27]:
import torch
import numpy as np

input = np.array(
[[[
    [1  ,1,0,0,0,1],
    [1  ,1,0,1,0,1],
    [0.5,1,1,1,0,0],
    [0  ,1,0,1,0,1],
    [0  ,1,1,0,0,1],
    [1  ,0,1,0,0,0],
]
],
    [[
    [1  ,1,0,0,0,1],
    [1  ,1,0,1,0,1],
    [0.5,1,1,1,0,0],
    [0  ,1,0,1,0,1],
    [0  ,1,1,0,0,1],
    [1  ,0,1,0,0,0],
]
]
]
)

kernel = np.transpose(np.array(
[[[
    [1,1,1,1,1,1.0],
    [1,1,1,1,1,1]
]]]),[0,1,2,3])

In [42]:
# One image of one feature map 6x6
# The kernel has 1 feature map out, 1 feature map in, 2 vectors of size 6
input.shape,kernel.shape

((2, 1, 6, 6), (1, 1, 2, 6))

In [43]:
input_torch = torch.FloatTensor(input)
kernel_torch = torch.FloatTensor(kernel)

In [44]:
import torch.nn.functional as F
import torch.nn
from torch.autograd import Variable
cnn = torch.nn.Conv2d(1,1,(1,6),bias=False)
cnn.weight.data.copy_(kernel_torch[:,:,0,:])
a = cnn.forward(Variable(input_torch))
print(a.size())

torch.Size([2, 1, 6, 1])


In [56]:
cnn2 = torch.nn.Conv2d(1,1,(6,1),bias=False)
cnn2.weight.data.copy_(kernel_torch[:,:,0,:])

b = cnn2.forward(Variable(input_torch))
print(b.size())

RuntimeError: The expanded size of the tensor (1) must match the existing size (6) at non-singleton dimension 3

In [14]:
a

tensor([[[[ 3.0000],
          [ 4.0000],
          [ 3.5000],
          [ 3.0000],
          [ 3.0000],
          [ 2.0000]]],


        [[[ 3.0000],
          [ 4.0000],
          [ 3.5000],
          [ 3.0000],
          [ 3.0000],
          [ 2.0000]]]])

In [15]:
torch.cat([a]*6,3)


tensor([[[[ 3.0000,  3.0000,  3.0000,  3.0000,  3.0000,  3.0000],
          [ 4.0000,  4.0000,  4.0000,  4.0000,  4.0000,  4.0000],
          [ 3.5000,  3.5000,  3.5000,  3.5000,  3.5000,  3.5000],
          [ 3.0000,  3.0000,  3.0000,  3.0000,  3.0000,  3.0000],
          [ 3.0000,  3.0000,  3.0000,  3.0000,  3.0000,  3.0000],
          [ 2.0000,  2.0000,  2.0000,  2.0000,  2.0000,  2.0000]]],


        [[[ 3.0000,  3.0000,  3.0000,  3.0000,  3.0000,  3.0000],
          [ 4.0000,  4.0000,  4.0000,  4.0000,  4.0000,  4.0000],
          [ 3.5000,  3.5000,  3.5000,  3.5000,  3.5000,  3.5000],
          [ 3.0000,  3.0000,  3.0000,  3.0000,  3.0000,  3.0000],
          [ 3.0000,  3.0000,  3.0000,  3.0000,  3.0000,  3.0000],
          [ 2.0000,  2.0000,  2.0000,  2.0000,  2.0000,  2.0000]]]])

In [16]:
b

NameError: name 'b' is not defined

In [17]:
torch.cat([b]*6,2)


NameError: name 'b' is not defined

In [ ]:
torch.cat([a]*6,3)+torch.cat([b]*6,2)


In [7]:
class E2EBlock(torch.nn.Module):
    '''E2Eblock.'''

    def __init__(self, in_planes, planes,example,bias=True):
        super(E2EBlock, self).__init__()
        self.d = example.size(3)
        self.cnn1 = torch.nn.Conv2d(in_planes,planes,(1,self.d),bias=bias)
        self.cnn2 = torch.nn.Conv2d(in_planes,planes,(self.d,1),bias=bias)

        
    def forward(self, x):
        a = self.cnn1(x)
        b = self.cnn2(x)
        return torch.cat([a]*self.d,3)+torch.cat([b]*self.d,2)



In [13]:
block = E2EBlock(1,1,input_torch,False)
block(Variable(input_torch))

tensor([[[[ 0.4799,  1.2642,  1.1106,  0.9420,  0.5945,  1.0179],
          [ 0.0978,  0.8821,  0.7285,  0.5599,  0.2124,  0.6358],
          [-0.5087,  0.2756,  0.1221, -0.0465, -0.3941,  0.0293],
          [ 0.1739,  0.9582,  0.8047,  0.6360,  0.2885,  0.7119],
          [ 0.2946,  1.0789,  0.9253,  0.7567,  0.4092,  0.8326],
          [-0.4521,  0.3322,  0.1786,  0.0100, -0.3375,  0.0859]]],


        [[[ 0.4799,  1.2642,  1.1106,  0.9420,  0.5945,  1.0179],
          [ 0.0978,  0.8821,  0.7285,  0.5599,  0.2124,  0.6358],
          [-0.5087,  0.2756,  0.1221, -0.0465, -0.3941,  0.0293],
          [ 0.1739,  0.9582,  0.8047,  0.6360,  0.2885,  0.7119],
          [ 0.2946,  1.0789,  0.9253,  0.7567,  0.4092,  0.8326],
          [-0.4521,  0.3322,  0.1786,  0.0100, -0.3375,  0.0859]]]])

In [ ]:
"""" KERAS MODEL 
model.add(E2E_conv(2,32,(2,n_feat),kernel_regularizer=reg,input_shape=(n_feat,n_feat,1),input_dtype='float32',data_format="channels_last"))
print("First layer output shape :"+str(model.output_shape))
model.add(LeakyReLU(alpha=0.33))
#print(model.output_shape)
model.add(E2E_conv(2,32,(2,n_feat),kernel_regularizer=reg,data_format="channels_last"))
print(model.output_shape)
model.add(LeakyReLU(alpha=0.33))
model.add(Convolution2D(64,(1,n_feat),kernel_regularizer=reg,data_format="channels_last"))
model.add(LeakyReLU(alpha=0.33))
model.add(Convolution2D(256,(n_feat,1),kernel_regularizer=reg,data_format="channels_last"))
model.add(LeakyReLU(alpha=0.33))
#print(model.output_shape)
model.add(Dropout(0.5))


model.add(Dense(128,kernel_regularizer=reg,kernel_initializer=kernel_init))
#print(model.output_shape)
model.add(LeakyReLU(alpha=0.33))
#print(model.output_shape)
model.add(Dropout(0.5))
model.add(Dense(30,kernel_regularizer=reg,kernel_initializer=kernel_init))
model.add(LeakyReLU(alpha=0.33))
#print(model.output_shape)
model.add(Dropout(0.5))
model.add(Dense(2,kernel_regularizer=reg,kernel_initializer=kernel_init))
model.add(Flatten())
model.add(LeakyReLU(alpha=0.33))
""""


In [77]:
F.dropout?

In [87]:
class BrainNetCNN(torch.nn.Module):
    def __init__(self, example, num_classes=10):
        super(BrainNetCNN, self).__init__()
        self.in_planes = example.size(1)
        self.d = example.size(3)
        
        self.e2econv1 = E2EBlock(1,32,example)
        self.e2econv2 = E2EBlock(32,64,example)
        self.E2N = torch.nn.Conv2d(64,1,(1,self.d))
        self.N2G = torch.nn.Conv2d(1,256,(self.d,1))
        self.dense1 = torch.nn.Linear(256,128)
        self.dense2 = torch.nn.Linear(128,30)
        self.dense3 = torch.nn.Linear(30,2)

    def forward(self, x):
        out = F.leaky_relu(self.e2econv1(x),negative_slope=0.33)
        out = F.leaky_relu(self.e2econv2(out),negative_slope=0.33) 
        out = F.leaky_relu(self.E2N(out),negative_slope=0.33)
        out = F.dropout(F.leaky_relu(self.N2G(out),negative_slope=0.33),p=0.5)
        out = out.view(out.size(0), -1)
        out = F.dropout(F.leaky_relu(self.dense1(out),negative_slope=0.33),p=0.5)
        out = F.dropout(F.leaky_relu(self.dense2(out),negative_slope=0.33),p=0.5)
        out = F.leaky_relu(self.dense3(out),negative_slope=0.33)
        
        return out


In [88]:
net = BrainNetCNN(input_torch)
net(Variable(input_torch))

tensor(1.00000e-02 *
       [[ 4.9636, -0.9249],
        [ 4.9636, -0.9249]])

refs : carlos , voir [ici](https://github.com/brain-bzh/MCNN/blob/master/proposed/pines_aux.py) et [ici](https://github.com/brain-bzh/MCNN/blob/master/proposed/cifar.py)


ref: BrainNetCNN avec données LSD77 [ici](https://github.com/nicofarr/BrainCNN_Vis/blob/master/BrainNetCNN.py)